In [ ]:
from flask import Flask, request

app = Flask(__name__)

FB_API_URL = 'https://graph.facebook.com/v2.6/me/messages'
VERIFY_TOKEN = '3MfNfMh1r7bEmT6bhoIDMVjO6y4a22InmbGLoMA1des='# <paste your verify token here>
PAGE_ACCESS_TOKEN = 'EAADDsxNtDVYBAJxxhzBZARCQSQq6p8gAYkZCiasedSnJrOUZCJG3CGrTZBPwSCXXmsLuuvFVzuwipMbh5nPOxTD3MbZAU0qMnBeMuY16cIyZBVInhG2NaHGf6Ny2kKfAPW2RzzjZBuWnkNIG9EjmjUSPLY6a0ZBQIU1JMPJRGh6nZCQZDZD'# paste your page access token here>"




def verify_webhook(req):
    if req.args.get("hub.verify_token") == VERIFY_TOKEN:
        return req.args.get("hub.challenge")
    else:
        return "incorrect"

key_words = {"racial_slurs" : ['ape', 'monkey', 'nigger', 'niger', 'negro', 'ape', 'gorilla', 'bounty', 'burrhead',
                'coon', 'crow', 'darky', 'darkie', 'alibaba', 'beaner', 'beaney', 'brownie',
                'charlie', 'chichi', 'cheechee', 'ching', 'chong', 'chink']}

# Define a dictionary of patterns
patterns = {}

# Iterate over the keywords dictionary
for intent, key in key_words.items():
    # Create regular expressions and compile them into pattern objects
    patterns[intent] = re.compile('|'.join(key))
    
intent = ["no_call", "immediate_threat", "medical_emergency", "accident", "fire"]

responses = {"racist_message" : ['You are the problem and they should probably call the police on you', 
                               "Let me guess, you are in fact mad at yourself for being a nasty racist",
                               "Start by respecting people"],
             "no_call" : ["stop wasting precious resources.",
                          "mind your own business!"],
             "immediate_threat" : ["please call 911 and try to find a spot where you are safe. help is on the way",
                                   "Call 911 immediately!"],
             "medical_emergency" : ["Please try to stay calm and call 911. Explain all the symptoms if you can.",
                                    "Call 911. If you are the person suffering, try to relax. If you are witnessing, try to help but get out of the way as soon as the help is here."],
             "accident" : ["Please call 911 and explain calmly the situation.",
                           "Call 911 and try to get everybody off the danger perimeter"],
             "fire" : ["Call 911 and follow fire procedure",
                       "I told you not to play with matches!"]}




# Define a function to find the intent of a message
def match_intent(message):
    for intent, pattern in patterns.items():
        if re.search(pattern, message) is not None:
            matched_intent = "racist_message"
        else:
            # Load the spacy model: nlp
            nlp = spacy.load('en')

            # Calculate the length of sentences
            n_sentences = 1

            # Calculate the dimensionality of nlp
            embedding_dim = 384

            # Initialize the array with zeros: X
            X = np.zeros((n_sentences, embedding_dim))

            # Iterate over the sentences
            doc = nlp(message)
            # Save the document's .vector attribute to the corresponding row in X
            X[:] = doc.vector

            pred = load_model.predict(X)
            matched_intent = intent[int(pred)]
    return matched_intent

    
# Define a respond function
def respond(message):
    
    if '\U0001F369' in message:
        send_message(sender, "ooooooooouh donuts")
    else:
        # Call the match_intent function
        intent = match_intent(message)
        # Fall back to the default response
        send_message(sender, random.choice(responses[intent]))

def is_user_message(message):
    """Check if the message is a message from the user"""
    return (message.get('message') and
            message['message'].get('text') and
            not message['message'].get("is_echo"))


@app.route("/webhook",methods=['GET','POST']) 
def listen():
    """This is the main function flask uses to 
    listen at the `/webhook` endpoint"""
    if request.method == 'GET':
        return verify_webhook(request)

    if request.method == 'POST':
        payload = request.json
        event = payload['entry'][0]['messaging']
        for x in event:
            if is_user_message(x):
                text = x['message']['text']
                sender_id = x['sender']['id']
                respond(sender_id, text)

        return "ok"
    

import requests

def send_message(recipient_id, text):
    """Send a response to Facebook"""
    payload = {
        'message': {
            'text': text
        },
        'recipient': {
            'id': recipient_id
        },
        'notification_type': 'regular'
    }

    auth = {
        'access_token': PAGE_ACCESS_TOKEN
    }

    response = requests.post(
        FB_API_URL,
        params=auth,
        json=payload
    )

    return response.json()